In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['FIP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('feline infectious peritonitis', 248),
 ('familial interstitial pneumonia', 14),
 ('fusion inhibitory peptide', 10),
 ('forward inner primer', 8),
 ('fipronil', 5),
 ('fungal immunomodulatory protein', 5),
 ('focal intestinal perforation', 4),
 ('f2 isoprostanes', 3)]

In [7]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [8]:
result = [grounding_map, names, pos_labels]

In [9]:
result

[{'f2 isoprostanes': 'CHEBI:CHEBI:142058',
  'familial interstitial pneumonia': 'MESH:D054988',
  'feline infectious peritonitis': 'MESH:D016766',
  'fipronil': 'CHEBI:CHEBI:5063',
  'focal intestinal perforation': 'MESH:D007416',
  'forward inner primer': 'ungrounded',
  'fungal immunomodulatory protein': 'ungrounded',
  'fusion inhibitory peptide': 'MESH:D065147'},
 {'CHEBI:CHEBI:142058': 'F2-isoprostane',
  'MESH:D054988': 'Idiopathic Interstitial Pneumonias',
  'MESH:D016766': 'Feline Infectious Peritonitis',
  'CHEBI:CHEBI:5063': 'fipronil',
  'MESH:D007416': 'Intestinal Perforation',
  'MESH:D065147': 'Viral Fusion Protein Inhibitors'},
 ['MESH:D007416', 'MESH:D016766', 'MESH:D054988']]

In [6]:
grounding_map, names, pos_labels = [{'f2 isoprostanes': 'CHEBI:CHEBI:142058',
  'familial interstitial pneumonia': 'MESH:D054988',
  'feline infectious peritonitis': 'MESH:D016766',
  'fipronil': 'CHEBI:CHEBI:5063',
  'focal intestinal perforation': 'MESH:D007416',
  'forward inner primer': 'ungrounded',
  'fungal immunomodulatory protein': 'ungrounded',
  'fusion inhibitory peptide': 'MESH:D065147'},
 {'CHEBI:CHEBI:142058': 'F2-isoprostane',
  'MESH:D054988': 'Idiopathic Interstitial Pneumonias',
  'MESH:D016766': 'Feline Infectious Peritonitis',
  'CHEBI:CHEBI:5063': 'fipronil',
  'MESH:D007416': 'Intestinal Perforation',
  'MESH:D065147': 'Viral Fusion Protein Inhibitors'},
 ['MESH:D065147', 'MESH:D016766', 'MESH:D054988', 'CHEBI:CHEBI:5063']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('MESH:D065147', 'MESH:D065147', 0)]

In [15]:
intersection2

[('MESH:D016766', 'MESH:D065147', 0),
 ('ungrounded', 'MESH:D065147', 0),
 ('MESH:D065147', 'MESH:D065147', 0),
 ('MESH:D054988', 'MESH:D065147', 0),
 ('MESH:D007416', 'MESH:D065147', 0),
 ('CHEBI:CHEBI:5063', 'MESH:D065147', 0),
 ('CHEBI:CHEBI:142058', 'MESH:D065147', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [36]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-03 03:32:50] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-03 03:32:56] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9820168902136116 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'MESH:D016766': 150,
  'ungrounded': 12,
  'MESH:D065147': 4,
  'MESH:D054988': 6,
  'MESH:D007416': 2,
  'CHEBI:CHEBI:5063': 5,
  'CHEBI:CHEBI:142058': 2},
 'f1': {'mean': 0.981055, 'std': 0.015782},
 'precision': {'mean': 0.968965, 'std': 0.020084},
 'recall': {'mean': 0.993548, 'std': 0.012903},
 'MESH:D007416': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:D054988': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'ungrounded': {'f1': {'mean': 0.826667, 'std': 0.149666},
  'pr': {'mean': 0.733333, 'std': 0.226078},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D065147': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'CHEBI:CHEBI:5063': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D016766': {'f1': {'mea

In [37]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [39]:
disamb.dump(model_name, results_path)

In [38]:
print(disamb.info())

Disambiguation model for FIP

Produces the disambiguations:
	F2-isoprostane	CHEBI:CHEBI:142058
	Feline Infectious Peritonitis*	MESH:D016766
	Idiopathic Interstitial Pneumonias*	MESH:D054988
	Intestinal Perforation	MESH:D007416
	Viral Fusion Protein Inhibitors*	MESH:D065147
	fipronil*	CHEBI:CHEBI:5063

Class level metrics:
--------------------
Grounding                         	Count	F1     
     Feline Infectious Peritonitis*	150	0.98688
                        Ungrounded	 12	0.82667
Idiopathic Interstitial Pneumonias*	  6	    0.8
                          fipronil*	  5	    1.0
   Viral Fusion Protein Inhibitors*	  4	    0.8
            Intestinal Perforation	  2	    0.4
                    F2-isoprostane	  2	    0.4

Weighted Metrics:
-----------------
	F1 score:	0.98202
	Precision:	0.97048
	Recall:		0.99394

* Positive labels
See Docstring for explanation



In [40]:
model_to_s3(disamb)